# Autoencoder based time-series explanation

In [ ]:
import os
import glob
import numpy as np
# from matplotlib import pyplot as plt
# from matplotlib import gridspec
# from skimage.transform import resize
# from tqdm import tqdm
from scipy import linalg
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
from scipy.interpolate import interp1d
import sklearn
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from tensorflow.keras.models import model_from_json
import platform
import sys
from math import sqrt
import matplotlib
SPINE_COLOR = 'gray'
from sklearn import preprocessing
import os
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
__file__ = '/home/s7thakur/ecresearch-shared/PhysioNet/resnet1d/resnet1d'
sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(__file__))))

from resnet1d.resnet1d import MyDataset, ResNet1D



In [ ]:

from utils.utils import reshape, label_encoding, calculate_metrics, smooth, norm, read_dataset_mitdb, torch_norm

In [ ]:
# import sys
# sys.path.append('/home/s7thakur/ecresearch-shared/compute1/sktime-dl')
# from sktime_dl.deeplearning.inceptiontime._classifier import InceptionTimeClassifier

In [ ]:
import sys
print(sys.version)

In [ ]:
sys.path.append('/home/s7thakur/ecresearch-shared/compute1/RISE/')
from utils_ts import read_dataset, reshape, label_encoding, calculate_metrics

In [ ]:
# from utils_ts import latexify

In [ ]:
import platform 
print(platform.node())

## Change code below to incorporate your *model* and *input processing*

### Load your model here:

In [ ]:
# datasets = ['PhalangesOutlinesCorrect', 'CinCECGTorso','ItalyPowerDemand', 'Trace', 'GunPoint', 'GunPointAgeSpan', 
#                             'Strawberry', 'ECGFiveDays', 'TwoLeadECG', 'Chinatown', 'DistalPhalanxOutlineCorrect']

# dataset = datasets[3]

# if platform.node() == "compute3.esg.uwaterloo.ca":
#     if os.listdir('/home/s7thakur/ecresearch-shared/compute1'):
#         filepath = '/home/s7thakur/ecresearch-shared/compute1'
    
#     else: 
#         print('ecresearch-shared is not mounted')
        
# elif platform.node() == "compute1.esg.uwaterloo.ca":
#     if not os.listdir('rhome/s7thakur/ecresearch-s7thakur/compute1'):
#         filepath = '/rhome/s7thakur/ecresearch-s7thakur/compute1'
#         print(filepath)
#     else: 
#         print('ecresearch-shared is not mounted')
    
# else: print('file path does not exist')

dataset_name='mitdb'
archive_name = 'PhysioNet'
filepath='/home/s7thakur/ecresearch-shared'
    
archive_dir=os.path.join(filepath,'data','PhysioNet')
model_dir = os.path.join(filepath, 'PhysioNet')


# dir = os.path.join('/rhome/s7thakur/MIRROR/logs/autoencoder/saliency/')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()


In [ ]:
device_str = "cuda:0"
device = torch.device(device_str if torch.cuda.is_available() else "cpu")
print(device)
model = ResNet1D(
    in_channels=1, 
    base_filters=128, 
    kernel_size=16, 
    stride=2, 
    groups=8, 
    n_block=8, 
    n_classes=len(np.unique(Y_train)), 
    downsample_gap=2, 
    increasefilter_gap=4, 
    use_do=False)

# summary(model, torch.zeros(1, 1, 360))
model.to(device)

# define optimzer
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# define checkpoint saved path
ckp_path = os.path.join(model_dir,"current_checkpoint_two_class.pt")

# load the saved checkpoint
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

print("model = ", model)
print("optimizer = ", optimizer)
print("start_epoch = ", start_epoch)
print("valid_loss_min = ", valid_loss_min)
print("valid_loss_min = {:.6f}".format(valid_loss_min))

model.eval()

## Load Dataset
### Input: $x \in \mathcal{R}^d$, Class: $y \in \mathcal{R}^c$

In [ ]:
def label2index(i):
#     m = {'N':0, 'S':1, 'V':2, 'F':3, 'Q':4} # uncomment for 5 classes
    m = {'N':0, 'S':0, 'V':1, 'F':0, 'Q':0} # uncomment for 2 classes
    return m[i]
batch_size=256
data = np.load(os.path.join(filepath, archive_name, 'data', 'mitdb_data.npy'))
label_str = np.load(os.path.join(filepath, archive_name, 'data', 'mitdb_group.npy'))

print(np.unique(label_str))
label = np.array([label2index(i) for i in label_str])

# make data
train_ind = np.load(os.path.join(filepath, archive_name, 'data', 'mitdb_train_ind.npy'))
test_ind = np.load(os.path.join(filepath, archive_name, 'data', 'mitdb_test_ind.npy'))
# data = preprocessing.scale(data, axis=1)

X_train = data[train_ind]
X_test = data[test_ind]
Y_train = label[train_ind]
Y_test = label[test_ind]

std_ = X_train.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
X_train = (X_train - X_train.mean(axis=1, keepdims=True)) / std_

std_ = X_test.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
X_test = (X_test - X_test.mean(axis=1, keepdims=True)) / std_

print(X_train.shape, Counter(Y_train))
print(X_test.shape, Counter(Y_test))
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)
print(X_train.shape, Counter(Y_train))
print(np.max(X_train), np.min(X_train))
# for i in range(20):
#     plt.figure()
#     idx = np.random.randint(X_train.shape[0])
#     title = '{}_{}'.format(Y_train[idx], idx)
#     plt.plot(X_train[idx])
#     plt.title(title)
#     plt.savefig('img/{0}.png'.format(title))
# exit()

# prepare loader
shuffle_idx = np.random.permutation(list(range(X_train.shape[0])))
X_train = X_train[shuffle_idx]
Y_train = Y_train[shuffle_idx]
X_train = np.expand_dims(X_train, 1)
X_test = np.expand_dims(X_test, 1)
dataset = MyDataset(X_train, Y_train)
# dataset_test = MyDataset(X_test, Y_test)
# print(X_train[1550:1600].shape, Y_train[1550:1600].shape, Counter(Y_train[1550:1600]))
dataset_test = MyDataset(X_train[1550:1600], Y_train[1550:1600])

dataloader = DataLoader(dataset, batch_size=batch_size, drop_last=False, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=1, drop_last=False, shuffle=False)


In [ ]:
print(len(dataloader_test))

In [ ]:
test_acc = 0.0
for samples, labels in dataloader_test:
    with torch.no_grad():
        samples, labels = samples.cuda(), labels.cuda()
        
#         print(samples.shape, labels.shape)
        output = model(samples)
        # calculate accuracy
        pred = torch.argmax(output, dim=1)
        correct = pred.eq(labels)
        test_acc += torch.mean(correct.float())
        # print('output {}'.format(output))
        # print('correct {}'.format(correct))
        # print('predict {}'.format(pred))

        # print('test Accuracy {}'.format(test_acc))
print(len(dataloader_test))
print('Accuracy of the network on {} test images: {}%'.format(len(labels), round(test_acc.item()*100.0/len(dataloader_test), 2)))



In [ ]:
input_shape = X_test.shape[1:]
    
    
N=len(dataloader_test)

In [ ]:
def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError

    if x.size < window_len:
        raise ValueError


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]

    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [ ]:

x_0=[]
x_1=[]
labels = np.unique(Y_train)
for samples, labels in dataloader_test:
    with torch.no_grad():
        samples, labels = samples.cuda(), labels.cuda()
        print(labels.cpu().numpy())
        print(samples.cpu().numpy()[0][0].shape)
        if labels.cpu().numpy()[-1] == 0:
            
            if len(x_0) ==5:
                continue
            else:
                x_0.append(samples.cpu().numpy()[0][0])
        elif labels.cpu().numpy()[-1] == 1:
            if len(x_1) ==5:
                continue
            else:
                x_1.append(samples.cpu().numpy()[0][0])
        if len(x_1)==5:
            break
print(np.array(x_1).shape)


In [ ]:
x_0 = np.array(x_0)
x_1 = np.array(x_1)

plt.figure()
for x in x_0:
#     print(x.shape)
    plt.plot(x_0.T, c='g')
for x in x_1:
    plt.plot(x_1.T,c='r')
plt.show()


In [ ]:
labels = np.unique(Y_train)
print(labels)

## Generate random samples ($s \in \mathcal{S}^d$) where $\mathcal{S} = \mathcal{N}(0,1)$ 

In [ ]:
# N = 500
# w_size=20 works in general
def gen_initial_mask(N, s,p=0.5):
    
#     print('(N,s)', N, s)
#     grid = np.random.randn(s, 1) 
    grids = np.empty((N, s))
    samples = np.random.normal(0,1, size=(N, s)) < p
#     samples = min + (samples * (max - min))
    samples = samples.astype('float32')
    for i in range(0, N):
        grids[i] = smooth(samples[i], 20)[0:s]
    return grids


def gen_initial_mask_binary(N, s,p=0.5,max_lobes=3, winsize=20,window='hamming'):
    
#     print('(N,s)', N, s)
    high = int(s/(s/max_lobes))
    win = int((winsize/100)*s)
    low = np.random.randint(1,high=high, size=N)
#     print(high, win)
    rng = np.random.default_rng()
    
    samples = np.concatenate((np.ones((N,low[0])),np.zeros((N,s-low[0]))),axis=1)
    for i,sample in enumerate(samples):
    
        rng.shuffle(sample)
        
        samples[i]=smooth(sample,win,window=window)[0:s]
        
    samples = samples.astype('float32')
    
    return samples

def gen_mask_binary(N, s,p=0.5,max_lobes=3, winsize=20,window='hamming'):
    
#     print('(N,s)', N, s)
    high = int(s/(s/max_lobes))
    win = int((winsize/100)*s)
    lobes = np.random.randint(1,high=high, size=N)
#     print(lobes)
    rng = np.random.default_rng()
    
    samples = np.concatenate((np.ones((N,lobes[0])),np.zeros((N,s-lobes[0]))),axis=1)
    for i,sample in enumerate(samples):
    
        rng.shuffle(sample)
        
        samples[i]=smooth(sample,win,window=window)[0:s]
    
    samples = samples.astype('float32')
    
    return samples


In [ ]:
20/6

In [ ]:
# samples = gen_initial_mask_binary(2,82,p=0.6,winsize=10,window='flat')
samples = gen_mask_binary(2,82,p=0.6,max_lobes=5,winsize=20,window='flat')

plt.figure()
for sample in samples:
    plt.plot(sample)
plt.show()

In [ ]:

s=100
print((s/10))
low = np.random.randint(2,int(s/30), 1)
print(5,low)
rng = np.random.default_rng()

samples = np.concatenate((np.ones(low[0]),np.zeros((s-low[0]))),axis=None)
print(samples)
rng.shuffle(samples)
print(samples)
plt.plot(smooth(samples,30))

In [ ]:
s=100
N=20
rng = np.random.default_rng()
print((s/20))
low = np.random.randint(2,high=int(s/30), size=N)
print(5,low)
samples = np.concatenate((np.ones((N,low[0])),np.zeros((N,s-low[0]))),axis=1)
print(samples.shape)
for i,sample in enumerate(samples):
#     print(sample.shape)
    rng.shuffle(sample)
    samples[i]=smooth(sample,40,window='hamming')[0:s]
#     print(sample)
plt.figure()
for sample in samples:
    plt.plot(sample)
plt.show()


## Algorithm to weigh the input samples based on relevance by randomly subsampling input 

In [ ]:

from sklearn.metrics import mutual_info_score
import sklearn

def norm(x):
    
    return  (x - np.mean(x))/np.std(x)

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N = 1000


    
def calc_mse(x, y, z, recon_pred, masked_preds, test, class_idx, lamb):
    x = x.reshape(x.shape[0], x.shape[1])
    y = y.reshape(y.shape[0], y.shape[1])
    grads_recon = np.gradient(y, axis=1)
    grads_masked = np.gradient(np.square(np.abs(z-test)), axis=1)
    mse1 =recon_preds.T.dot(( np.gradient(x, axis=1))*lamb)
    mse2 = masked_preds.T.dot((grads_masked)*lamb)
    mse=mse2
    
#     plt.figure()
#     for ms in mse:
#         print(ms.shape)
#         plt.plot(ms)
#     plt.show()
    
    
    return mse



if x_train.shape[0] < x_test.shape[0]:
    test = x_train[0:50]
    labels = y_train[0:50]
    
else:
    test = x_test[0:50]
    labels = y_test[0:50]
    
# print(labels)
n_classes=len(nb_labels)
print(n_classes)
sal = np.empty((test.shape[0], test.shape[1],n_classes))   
# sal = np.empty((test.shape[0], test.shape[1],len(nb_labels)))   
for x, test_input in enumerate(test):
    
    samples = gen_initial_mask(N, test.shape[1], np.mean(test),np.std(test))
    sample = samples[0]
    samples = samples.reshape(samples.shape[0], samples.shape[1],1)
    
    test_input = (test_input - np.mean(test_input))/np.std(test_input)
#     print('Data after normalization',test_input.shape)
    masked_inputs = samples*test_input + samples
    
    masked_inputs = (masked_inputs - np.mean(masked_inputs))/np.std(masked_inputs)
#     print('samples', masked_inputs.shape)
    
    true_pred = model.predict(test_input.reshape(1, test.shape[1],1))
    true_preds = np.tile(true_pred, (N,1))
    class_idx = np.argmax(true_pred)
    
    classes = np.unique(np.concatenate((y_train, y_test), axis=0))
    masked_preds = model.predict(masked_inputs)
    plt.figure()
    for inp in masked_inputs[0:30]:
        plt.plot(inp)
    plt.show()
    test_inputs = np.tile(test_input[:,0], (N, 1))
#     test_inputs = test_inputs.reshape(N, test.shape[1],1)
#     print(masked_inputs.shape)
    recon = autoencoder.predict(masked_inputs)
    recon_preds = model.predict(recon.reshape(N, test.shape[1], 1))
    print(recon_preds.shape, masked_inputs.shape, recon.shape, test_inputs.shape)
    mse = calc_mse(recon, samples, masked_inputs[:,:,0], recon_preds, masked_preds, test_inputs, class_idx, 0.005)
#     sal[x] = recon_preds[:,class_idx].T.dot(mse*masked_inputs.reshape(masked_inputs.shape[0], masked_inputs.shape[1])).reshape(test.shape[1],1)
#     sal[x] = recon_preds[:,class_idx].T.dot(mse).reshape(test.shape[1],1)
    sal[x] = mse.reshape(test.shape[1],n_classes)
    sal[x]=mse.T
    print(sal[x].shape)
#     sal[x] = np.sum(mse*masked_inputs.reshape(masked_inputs.shape[0], masked_inputs.shape[1]), axis=0).reshape(sal.shape[1],1)
#     sal[x] = np.sum((recon.reshape(recon.shape[0], recon.shape[1],1) - test_input)*(recon_preds[:,class_idx])*lamb)
#     sal[x] = np.sum( (np.abs((samples - recon)))*lamb + recon*(1-lamb))
    
#     sal[x] = np.sum(mse)
#         print(recon[i].shape, x_train[0].shape, mse.shape)
    plt.figure()
#     plt.plot(sal[x,:,0])
#     plt.plot(sal[x,:,1])
    
    for s in sal[x].T:
        plt.plot(np.abs(s))
    plt.show()

# np.save("/rhome/s7thakur/MIRROR/scores/UCI-" + dataset +"-MIRROR-SaliencyScore.npy",sal)

In [ ]:
np.unique(labels)

In [ ]:
# assign score using optimization 
import numpy as np
# from autograd import grad
from numpy import linalg as LA
# from autograd import elementwise_grad as egrad
from sklearn.metrics import mutual_info_score
import sklearn

def norm(x):
    
    return  (x - np.mean(x))/np.std(x)

def minmax(x):
    
    return (x-np.min(x))/(np.max(x)-np.min(x))

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N=100
R=20
lamb1=0.9
lamb2=0.999
lr=1e-2
ep=1e-8
random_seed=0    
np.random.seed(random_seed)


if x_train.shape[0] < x_test.shape[0]:
    test = x_train[0:50]
    labels = y_train[0:50]
    
else:
    test = x_test[0:50]
    labels = y_test[0:50]
    
# print(labels)

def cost(M):
    
    y = M*(np.sqrt(masked_pred[0,class_idx])**2)
    
    return y

def scale(sal):
#     print(sal.shape)
    for i,sa in enumerate(sal):
#         print(i, sa.shape)
        minimum = np.min(sa)
        sa = np.subtract(sa , minimum)
        sa = np.divide(sa, np.max(sa, axis=0))
        sal[i,:] = sa * 100
#     print(sal.shape)
    return sal


n_classes = len(np.unique(nb_labels))
# initialize moment and velocity vectors for the target class
sal = np.ones((test.shape[0], test.shape[1],n_classes))      

for x,test_input in enumerate(test):
    
    sal_itrs = np.empty((R, test.shape[1],n_classes))   
    preds = np.empty((R, 1,n_classes))      
    
#     print(sal_itrs.shape)
    for i in range(0,R):
        


#         M = gen_initial_mask(100, test.shape[1],0.5)
        M=gen_initial_mask_binary(N,test.shape[1],p=0.6,window='flat')
#         M = gen_initial_mask_binary(100, test.shape[1],0.5)
        
#         plt.figure()
#         for inp in M[0:30]:
#             plt.plot(inp)
#         plt.show()
        
        M = M.reshape(M.shape[0], M.shape[1],1)
        M = minmax(M)
        test_input = minmax(test_input)
#         masked_inputs = M
        masked_inputs = M*test_input
        test_inputs = np.tile(test_input[:,0], (N, 1))
        test_inputs = test_inputs.reshape(test_inputs.shape[0],test_inputs.shape[1],1)

        true_preds = model.predict(test_inputs)
        masked_preds = model.predict(masked_inputs)
        unmasked_preds = model.predict(1-masked_inputs)
        
    #     mask_grads = np.gradient(masked_inputs[:,:,0][0:30],axis=1)



    #     plt.figure()
    #     for inp in mask_grads:
    #         plt.plot(1-inp)
    #     plt.show()
    #     print('masked input shape:', masked_inputs[:,:,0][0:30].shape)
        grad_on = ((np.gradient(norm(masked_inputs[:,:,0]),axis=1)))
                        
        grad_off = (np.abs(np.gradient(norm(1-masked_inputs[:,:,0]),axis=1)))
        
        plt.figure()
        for inp in grad_on[0:30]:
            plt.plot(inp)
        plt.show()
        
        masked_preds = model.predict(grad_on)
        unmasked_preds = model.predict(grad_off)
        
        M_grads_on= (masked_preds).T.dot((norm(grad_on)))*lr  
        M_grads_off= (1-masked_preds).T.dot((norm(grad_off)))*(lr)
        M_grads_anti= (unmasked_preds).T.dot((norm(grad_off)))*(lr)
        
#         + ep*LA.norm(np.abs(np.gradient(norm(masked_inputs[:,:,0]),axis=1)))
                           
#         + np.square(1-masked_preds).T.dot(norm(np.abs(np.gradient((1-masked_inputs[:,:,0]),axis=1))))*(1-lr)
    #     + np.square(1-masked_preds).T.dot(norm(np.abs(np.gradient((1-masked_inputs[:,:,0]),axis=1))))*(1-lr)
    #     M_grads= np.gradient(masked_preds.T.dot(norm(masked_inputs[:,:,0])),axis=1)*lr + np.gradient((1-masked_preds).T.dot(norm((1-masked_inputs[:,:,0]))),axis=1)*(1-lr)
    #     M_grads= np.square(np.gradient(np.square(masked_preds).T.dot(norm(masked_inputs[:,:,0])),axis=1))*lr  + np.square(np.gradient(np.square(1-masked_preds).T.dot(norm((1-masked_inputs[:,:,0]))),axis=1))*(1-lr)


#         dists = np.square((masked_preds).T.dot(((norm(masked_inputs[:,:,0])-norm(test_inputs[:,:,0])))))
#         dists = np.einsum("ij, jk->ik", M_grads_preds, norm(M_grads))
#         dists = M_grads.T.dot(M_grads_preds)
        temp_on=((M_grads_on))
        temp_off=((M_grads_off))
        unmasked = ((M_grads_anti))
#         print(unmasked.shape, temp_on.shape)
        print(np.mean(masked_preds,axis=0),np.mean(masked_preds,axis=0).shape)
    
        sal_itrs[i] = (((temp_on)).T)/N        
        
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_on:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('ON gradient with confidence score, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
        
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_off:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('OFF, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
#         plt.figure()
#         plt.plot(test_input,c='black')
#         for sa in unmasked:
#             plt.plot(norm(sa))
#         plt.legend(np.concatenate((['x'],nb_labels),axis=None))

#         plt.title('Unmasked, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
#         plt.show()
        
#         plt.figure()
#         plt.plot(test_input,c='black')
#         for sa in sal_itrs[i].T:
#             plt.plot((sa))
#         plt.legend(np.concatenate((['x'],nb_labels),axis=None))

#         plt.title('Saliency iteration, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
#         plt.show()
        
            
    print('sal_itrs shape',sal_itrs.shape)
    
#     temp = np.zeros((sal_itrs.shape[1],sal_itrs.shape[2]))
    
#     for sa in sal_itrs:
#         print('iterations')
# #         print(sa.shape)
        
#         print(sa.shape)
#         temp = temp + norm(sa)
#         print(temp.shape)
        
#         plt.figure()
#         for s in temp.T:
#             print('class specific sal')
#             plt.plot(norm(s))

#         plt.show()
        
#     print(temp)
#     print(np.sum(norm(np.abs(np.square(np.sum(sal_itrs, axis=0)))),axis=0).shape)
    temp = np.einsum('ijk->jk',sal_itrs)
#     for t in sal_itrs:
#         sal[x] = sal[x]+scale(t)
#     s=model.predict(temp.reshape(temp.shape[1],temp.shape[0],1))
    
    
#     temp = np.einsum('ij,ik->ikj',s, temp.T)
#     temp = np.einsum('ikj->ik',temp)
    
    
#     print(temp.shape)
    sal[x] = np.abs(temp)/R
    
    plt.figure()
    plt.plot(test_input,c='black')
    for sa in sal[x].T:
        plt.plot((norm(sa)))
    plt.legend(np.concatenate((['x'],nb_labels),axis=None))
    
    plt.title('Average Saliency, Predicted Class = {}'.format(np.argmax(true_preds[0])))
    plt.show()

In [ ]:
# filter masks with low confidence before averaging

# assign score using optimization 
import numpy as np
# from autograd import grad
from numpy import linalg as LA
# from autograd import elementwise_grad as egrad
from sklearn.metrics import mutual_info_score
import sklearn

def norm(x):
    
    return  (x - np.mean(x))/np.std(x)

def minmax(x):
    
    return (x-np.min(x))/(np.max(x)-np.min(x))

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N=100
R=20
lamb1=0.9
lamb2=0.999
lr=1e-2
ep=1e-8
random_seed=0    
np.random.seed(random_seed)


if x_train.shape[0] < x_test.shape[0]:
    test = x_train[0:50]
    labels = y_train[0:50]
    
else:
    test = x_test[0:50]
    labels = y_test[0:50]
    
# print(labels)

def cost(M):
    
    y = M*(np.sqrt(masked_pred[0,class_idx])**2)
    
    return y

def scale(sal):
#     print(sal.shape)
    for i,sa in enumerate(sal):
#         print(i, sa.shape)
        minimum = np.min(sa)
        sa = np.subtract(sa , minimum)
        sa = np.divide(sa, np.max(sa, axis=0))
        sal[i,:] = sa * 100
#     print(sal.shape)
    return sal


n_classes = len(np.unique(nb_labels))
# initialize moment and velocity vectors for the target class
sal = np.ones((test.shape[0], test.shape[1],n_classes))      

for x,test_input in enumerate(test):
    
    sal_itrs = np.empty((R, test.shape[1],n_classes))   
    preds = np.empty((R, 1,n_classes))      
    
#     print(sal_itrs.shape)
    for i in range(0,R):
        


#         M = gen_initial_mask(100, test.shape[1],0.5)
        M=gen_initial_mask_binary(N,test.shape[1],p=0.6,window='flat')
#         M = gen_initial_mask_binary(100, test.shape[1],0.5)
        
#         plt.figure()
#         for inp in M[0:30]:
#             plt.plot(inp)
#         plt.show()
        
        M = M.reshape(M.shape[0], M.shape[1],1)
        M = minmax(M)
        test_input = minmax(test_input)
#         masked_inputs = M
        masked_inputs = M*test_input
#         test_inputs = np.tile(test_input[:,0], (N, 1))
#         test_inputs = test_inputs.reshape(test_inputs.shape[0],test_inputs.shape[1],1)

        true_preds = model.predict(test_input.reshape(1,test_input.shape[0],1))
        masked_preds = model.predict(masked_inputs)
        unmasked_preds = model.predict(1-masked_inputs)
        true_class = np.argmax(true_preds)
        
    #     mask_grads = np.gradient(masked_inputs[:,:,0][0:30],axis=1)



    #     plt.figure()
    #     for inp in mask_grads:
    #         plt.plot(1-inp)
    #     plt.show()
    #     print('masked input shape:', masked_inputs[:,:,0][0:30].shape)
        grad_on = (np.abs(np.gradient(norm(masked_inputs[:,:,0]),axis=1)))
                        
        grad_off = (np.abs(np.gradient(norm(1-masked_inputs[:,:,0]),axis=1)))
        
        plt.figure()
        for inp in grad_on[0:30]:
            plt.plot(inp)
        plt.show()
        masked_preds = model.predict(grad_on)
        unmasked_preds = model.predict(grad_off)
        # check confidence scores
        include = np.where(masked_preds[:,true_class] > 0.7)
        print(include)
        masked_preds = masked_preds[include]
        print(true_preds,'pred class', true_class, 'true class',labels[x])
        
        grad_on = grad_on[include]
#         grad_off = grad_off[include]
        # filter masks with low confidence scores
        
        # multiply the masks with confidence scores and sum over the masks to get a final saliency 
        M_grads_on= np.square(masked_preds).T.dot((minmax(grad_on)))*lr  
#         M_grads_off= (1-masked_preds).T.dot((norm(grad_off)))*(lr)
        M_grads_anti= (unmasked_preds).T.dot((norm(grad_off)))*(lr)
        

        temp_on=((M_grads_on))
#         temp_off=((M_grads_off))
        unmasked = ((M_grads_anti))
    
        sal_itrs[i] = (((temp_on)).T)/N        
        
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_on:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('ON gradient with confidence score, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
        
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_off:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('OFF, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
#         plt.figure()
#         plt.plot(test_input,c='black')
#         for sa in unmasked:
#             plt.plot(norm(sa))
#         plt.legend(np.concatenate((['x'],nb_labels),axis=None))

#         plt.title('Unmasked, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
#         plt.show()
        
#         plt.figure()
#         plt.plot(test_input,c='black')
#         for sa in sal_itrs[i].T:
#             plt.plot((sa))
#         plt.legend(np.concatenate((['x'],nb_labels),axis=None))

#         plt.title('Saliency iteration, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
#         plt.show()
        
            
    print('sal_itrs shape',sal_itrs.shape)
    
#     temp = np.zeros((sal_itrs.shape[1],sal_itrs.shape[2]))
    
#     for sa in sal_itrs:
#         print('iterations')
# #         print(sa.shape)
        
#         print(sa.shape)
#         temp = temp + norm(sa)
#         print(temp.shape)
        
#         plt.figure()
#         for s in temp.T:
#             print('class specific sal')
#             plt.plot(norm(s))

#         plt.show()
        
#     print(temp)
#     print(np.sum(norm(np.abs(np.square(np.sum(sal_itrs, axis=0)))),axis=0).shape)
    temp = np.einsum('ijk->jk',sal_itrs)

    sal[x] = np.abs(temp)/R
    
    plt.figure()
    plt.plot(test_input,c='black')
    for sa in sal[x].T:
        plt.plot((norm(sa)))
    plt.legend(np.concatenate((['x'],nb_labels),axis=None))
    
    plt.title('Average Saliency, Predicted Class = {}'.format(np.argmax(true_preds[0])))
    plt.show()

In [ ]:
# filter masks with low confidence before averaging

# assign score using optimization 
import numpy as np
# from autograd import grad
from numpy import linalg as LA
# from autograd import elementwise_grad as egrad
from sklearn.metrics import mutual_info_score
import sklearn
import seaborn as sns
# import plotly.io as pio
# pio.renderers.default="jupyterlab"
# import plotly.figure_factory as ff
def norm(x):
    
    return  (x - np.mean(x))/np.std(x)

def minmax(x):
    
    return (x-np.min(x))/(np.max(x)-np.min(x))

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N=100
R=20
lamb1=0.9
lamb2=0.999
lr=1e-2
ep=1e-8
random_seed=0    
np.random.seed(random_seed)


if x_train.shape[0] < x_test.shape[0]:
    test = x_train[0:50]
    labels = y_train[0:50]
    
else:
    test = x_test[0:50]
    labels = y_test[0:50]
    
# print(labels)

def cost(M):
    
    y = M*(np.sqrt(masked_pred[0,class_idx])**2)
    
    return y

def scale(sal):
#     print(sal.shape)
    for i,sa in enumerate(sal):
#         print(i, sa.shape)
        minimum = np.min(sa)
        sa = np.subtract(sa , minimum)
        sa = np.divide(sa, np.max(sa, axis=0))
        sal[i,:] = sa * 100
#     print(sal.shape)
    return sal

def plot_score_densities(preds):
    preds = norm(preds)
    df = pd.DataFrame(preds, columns=list(map(str, np.arange(0,preds.shape[1]))))
    sns.displot(df,kind='ecdf')
    plt.show()
#     fig =  ff.create_distplot([df[c] for c in df.columns],df.columns, bin_size=0.25)
#     fig.update_layout(title_text='Density plot of confidence scores')
#     fig.show()
    
            

n_classes = len(np.unique(nb_labels))
print(n_classes)
# initialize moment and velocity vectors for the target class
sal = np.ones((test.shape[0], test.shape[1],n_classes))      
masked_preds_inputs = np.empty((test.shape[0],N,n_classes))
for x,test_input in enumerate(test):
    j=0
    
    sal_itrs = np.empty((R, test.shape[1],n_classes))   
    preds = np.empty((R, 1,n_classes))      
    temp_masked_preds = np.empty((R, N, n_classes))
#     print(sal_itrs.shape)
    for i in range(0,R):
        
        M=gen_initial_mask_binary(N,test.shape[1],p=0.6, max_lobes=3,winsize=20,window='flat')
        M_noise=gen_initial_mask(N,test.shape[1],p=0.6)
        
        M = M.reshape(M.shape[0], M.shape[1],1)
        M_noise = M_noise.reshape(M_noise.shape[0], M_noise.shape[1],1)
        
        M = norm(M)
        M = M*M_noise
        
        test_input = norm(test_input)
        print(M.shape, test_input.shape)
        masked_inputs = norm(M*test_input)

        true_preds = model.predict(test_input.reshape(1,test_input.shape[0],1))
        masked_preds = model.predict(masked_inputs)
        unmasked_preds = model.predict(1-masked_inputs)
        true_class = np.argmax(true_preds)
        plt.figure()
        for c in np.unique(nb_labels):
            print(masked_preds.shape)
            plt.plot(masked_preds[:,c])
        plt.legend(nb_labels)
        plt.show()
        
        plot_score_densities(masked_preds)
        
        print('true class',labels[x],'pred class',true_class, 'conf scores',true_preds)
        print('masked pred',np.argmax(masked_preds[2]))


        grad_on = ((np.gradient(norm(masked_inputs[:,:,0]),axis=1)))
                        
        grad_off = ((np.gradient(norm(1-masked_inputs[:,:,0]),axis=1)))
        
        plt.figure()
        for inp in masked_inputs[0:30]:
            plt.plot(inp)
        plt.show()
#         masked_preds = model.predict(grad_on)
#         unmasked_preds = model.predict(grad_off)
#         plot_score_densities(masked_preds)
#         check confidence scores
#         include = np.where(masked_preds[:,true_class] > 0.5)
#         print(include)
#         print(masked_preds)
#         masked_preds = masked_preds[include]
#         print(true_preds,'pred class', true_class, 'true class',labels[x],'masked pred',np.argmax(masked_preds,axis=1))
        
#         filter masks with low confidence scores
#         grad_on = grad_on[include]
        
#         multiply the masks with confidence scores and sum over the masks to get a final saliency 
        M_grads_on= (masked_preds).T.dot(((M[:,:,0])))*lr  
        M_grads_anti= (unmasked_preds).T.dot(((M[:,:,0])))*(lr)
        

        temp_on=((M_grads_on))
        temp_off = ((M_grads_anti))
        temp_masked_preds[i]=masked_preds
        sal_itrs[i] = (((temp_on)).T)/N       
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_on:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('ON gradient with confidence score, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
        
        
        plt.figure()
        plt.plot(test_input,c='black')
        for sa in temp_off:
            plt.plot(norm(sa))
        plt.legend(np.concatenate((['x'],nb_labels),axis=None))

        plt.title('OFF, Predicted Class = {},iteration {}'.format(np.argmax(true_preds[0]), i))
        plt.show()
        
        
            
#     print('sal_itrs shape',sal_itrs.shape)
    

    temp = np.einsum('ijk->jk',sal_itrs)
    
    print(np.mean(temp_masked_preds, axis=0).shape)
    masked_preds_inputs[x]=np.mean(temp_masked_preds, axis=0)
    sal[x] = (temp/R)
    j=j+1
    
    plt.figure()
    plt.plot(test_input,c='black')
    for sa in sal[x].T:
        plt.plot((norm(sa)))
    plt.legend(np.concatenate((['x'],nb_labels),axis=None))
    
    plt.title('Average Saliency, Predicted Class = {}'.format(np.argmax(true_preds[0])))
    plt.show()

In [ ]:
labels = np.unique(Y_train)

classes = labels

# N = X_test.shape[0]
N=len(dataloader_test)
# bins = 10
batch=300
epochs=20
lamb1=0.9
lamb2=0.999
lr=1e-2
ep=1e-8
random_seed=0    
np.random.seed(random_seed)

if len(input_shape) == 2:

    ndim = input_shape[0] if input_shape[1]<input_shape[0] else input_shape[1]
    
    
    
    
# if int((6/10)*self.ndim) <= self.window_size: return
# print(int((6/10)*self.ndim) , self.window_size)

n_classes = len(classes)

sal = np.ones((N, ndim, n_classes))      
print(sal.shape)
# total_preds = np.empty((self.N, self.epochs, self.batch, n_classes))
# total_true_preds = np.empty((self.N, self.epochs, self.batch, n_classes))


# print(self.inp_shape, len(self.test))

test_acc = 0.0
for x, row in enumerate(dataloader_test):

    with torch.no_grad():

        sample, label = row
        sample, label = sample.cuda(), label.cuda()

#         start = time.time()

        sal_itrs = np.empty((epochs, ndim, n_classes))   
#         print(sal_itrs.shape)
#         preds = np.empty((self.epochs, 1,n_classes))      
#         temp_masked_preds = np.empty((self.epochs, self.batch, n_classes))
#         temp_true_preds = np.empty((self.epochs, self.batch, n_classes))
        # print(sal_itrs.shape, temp_masked_preds.shape)
        j=0
        for i in range(0, epochs):


            M=gen_initial_mask_binary(batch,ndim,p=0.6, max_lobes=3,winsize=20,window='flat')
            M_noise=gen_initial_mask(batch,ndim,p=0.6)
#             print(M.shape, M_noise.shape)
#             M=gen_random_binary_mask_samples()
#             M_noise=get_gaussian_noise_samples()


            M = M.reshape(batch, *input_shape)
            M_noise = M_noise.reshape(batch, *input_shape)

            M = norm(M)
            M = M*M_noise
            M = torch.from_numpy(M).type(torch.cuda.FloatTensor).cuda()
            
            # print(M.shape)
            # print(M)
            # test_input = norm(samples)
            # print(sample.cpu().numpy())
            masked_inputs = torch_norm(M*sample)
            
            # print(masked_inputs)
            # masked_inputs = masked_inputs
            # print(masked_inputs.shape)

            # print(masked_inputs)


            # print(samples.shape, labels.shape)
            true_preds = model(sample)
            # calculate accuracy

            masked_preds = model(masked_inputs)
            unmasked_preds = model(1-masked_inputs)

            # print(masked_preds.size(), M[:,0,:].size())
            M_grads_on= torch.mul(torch.matmul(masked_preds.T, M[:,0,:]),torch.tensor(lr).cuda())
            M_grads_anti= torch.mul(torch.matmul(unmasked_preds.T, M[:,0,:]),torch.tensor(lr).cuda())
            # print(M_grads_on.size())
            true_preds = true_preds.cpu().numpy()
            masked_preds = masked_preds.cpu().numpy()
            M_grads_on = M_grads_on.cpu().numpy()
            M_grads_anti = M_grads_anti.cpu().numpy()

            temp_on= M_grads_on
            temp_off = M_grads_anti
#             temp_masked_preds[i]=masked_preds
#             temp_true_preds[i]=true_preds
            sal_itrs[i] = (temp_on.T)/batch    
            # print(sal_itrs.shape)


        temp = np.einsum('ijk->jk',sal_itrs)
        # print(temp.shape)

        # print(np.mean(temp_masked_preds, axis=0).shape)

        # masked_preds_inputs[x]=np.mean(temp_masked_preds, axis=0)
        sal[x] = (temp/epochs)

        plt.figure()
        plt.plot(sample.cpu().numpy()[0][0].T,c='black')
        for sa in sal[x].T:
            plt.plot((norm(sa)))
        plt.legend(np.concatenate((['x'],classes),axis=None))

        plt.title('Average Saliency')
        plt.show()


In [ ]:
sal.shape

In [ ]:
# assign score using optimization 
import numpy as np
# from autograd import grad
from numpy import linalg as LA
# from autograd import elementwise_grad as egrad
from sklearn.metrics import mutual_info_score
import sklearn

def norm(x):
    
    return  (x - np.mean(x))/np.std(x)

def minmax(x):
    
    return (x-np.min(x))/(np.max(x)-np.min(x))

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N=500
lamb1=0.9
lamb2=0.999
lr=1e-2
ep=1e-8
random_seed=0    
np.random.seed(random_seed)


if x_train.shape[0] < x_test.shape[0]:
    test = x_train[0:50]
    labels = y_train[0:50]
    
else:
    test = x_test[0:50]
    labels = y_test[0:50]
    
# print(labels)

def cost( pred):
    
    y = (np.sqrt(pred)**2)
    
    return y

def scale(sal):
    
    minimum = np.min(sal, axis=1)
    sal = np.subtract(sal , minimum)
    sal = np.divide(sal, np.max(sal, axis=1))
    sal = sal * 100
    return sal


n_classes = len(np.unique(nb_labels))
# initialize moment and velocity vectors for the target class
sal = np.empty((test.shape[0], test.shape[1],n_classes))      

for x,test_input in enumerate(test):
    
    

        print(test_input.shape)
        test_input = norm(test_input)
        
        test_input = test_input.reshape(1,test_input.shape[0],1)

        true_pred = model.predict(test_input)
        true_class = np.argmax(true_pred)
        print('true class',true_class)
        M = gen_initial_mask(1, test_input.shape[1],0.5)
        M = M.reshape(test_input.shape)
        
        M = M*(test_input)+M**2
        M = norm(M)
        plt.figure()
        plt.plot(M[0])
        plt.show()
        print(M.shape)
        masked_pred = model.predict(M)
        
        p_cur = masked_pred[0, true_class]
        p_prev = 0.0
        p_delta = 0.0
        sal_prev = M
        for i in range(0,N):
        
            
            masked_pred = model.predict(sal_prev)
            p_cur = masked_pred[0,true_class]
            print('masked pred', p_cur)
            
            if i == 1:
                
                sal_cur= (p_cur)*((np.abs(np.gradient(norm(M),axis=1)))) *lr
#                 + lr*LA.norm((np.abs(np.gradient(norm(M),axis=1))),'fro')  
            else:
                sal_cur= (p_cur)*((np.abs(np.gradient(norm(sal_prev),axis=1))))*(1-lr)  
#                 + (1-lr)*LA.norm((np.abs(np.gradient(norm(sal_prev),axis=1))),'fro')
                
            print(sal_cur.shape)
            if((p_cur - p_prev) > p_delta):
                ids_pos = np.where(np.abs(sal_prev[0,:,0] - sal_cur[0,:,0])>0)    
#                 print(ids_pos)
                sal_cur[0,ids_pos,0] = sal_cur[0,ids_pos,0] + cost(p_cur)*lr

            elif((p_cur-p_prev) <= p_delta ):
                ids_neg = np.where(np.abs(sal_prev[0,:,0]-sal_cur[0,:,0])>0)
                sal_cur[0,ids_neg,0] = sal_cur[0,ids_neg,0] - cost(p_cur)*(np.abs(sal_prev-sal_cur))[0,ids_neg,0]*lr


#             elif(p_delta-ep <=(p_cur-p_prev) <= p_delta+ep):
                

            
            p_prev = p_cur
            sal_prev = norm(sal_cur)
            
            plt.figure()
            plt.plot(test_input[0],c='black')
            for sa in sal_prev.T:
                plt.plot(norm(sa))
            # add legend

            plt.show()
        sal[x] = (sal_prev)/N
        
#         plt.figure()
#         plt.plot(test_input,c='black')
#         for sa in sal[x].T:
#             plt.plot(norm(sa))
#         # add legend

#         plt.title('Average Saliency, Predicted Class = {}'.format(np.argmax(true_preds[0])))
#         plt.show()
        

In [ ]:
if x_train.shape[0] < x_test.shape[0]:
        test = x_train
        labels = y_train

# else:
#     test = x_test[0:100]
#     labels = y_test[0:100]

In [ ]:
# using clipping on masked inputs (not used), saliency using only masked inputs
from sklearn.metrics import mutual_info_score
import sklearn

def norm(x):
    
    return  (x - np.mean(x))/np.std(x)
def minmax(x):
    
    return  (x - np.min(x))/(np.max(x)-np.min(x))

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))
    
# bins = 10
N = 2000

def calc_mse( y, z, masked_preds, true_preds, test,  class_idx, lamb):
#     grads_masked = np.gradient(z, axis=1)
    grads_masked = np.gradient(((z)), axis=1)
    mse1 = masked_preds.T.dot(grads_masked*lamb)
    mse2 = (masked_preds-true_preds).T.dot((np.abs((norm(z)-norm(test)))))
    
    mse=((minmax(np.abs(mse1))*minmax(np.abs(mse2))))
#     mse=((mse1))
    
    plt.figure()
    
    plt.plot((minmax(mse2).T),c='orange')
    plt.plot((minmax(mse1).T),c='green')
    plt.plot((minmax(mse).T),c='red')
    
    plt.show()
    

#     plt.figure()
#     for inp in grads[0:20]:
#         plt.plot(inp)
#     plt.show()
    
    
    return mse


def explain(train,test,y_test,y_train, nb_labels, model):

    

    # print(labels)
    n_classes=len(nb_labels)
    sal = np.empty((test.shape[0], test.shape[1],n_classes))   
    for x, test_input in enumerate(test):

        samples = gen_initial_mask(N, test.shape[1], np.mean(test_input), np.std(test_input))
        samples = samples.reshape(samples.shape[0], samples.shape[1],1)

        masked_inputs = samples*test_input+samples

        true_pred = model.predict(test_input.reshape(1, test.shape[1],1))
        true_preds = np.tile(true_pred, (N,1))
        class_idx = np.argmax(true_pred)

        classes = np.unique(np.concatenate((y_train, y_test), axis=0))
        masked_preds = model.predict(masked_inputs)
        plt.figure()
        for inp in masked_inputs[0:30]:
            plt.plot(inp)
        plt.show()
        test_inputs = np.tile(test_input[:,0], (N, 1))
    #     recon = autoencoder.predict(masked_inputs)
    #     recon_preds = model.predict(recon.reshape(N, test.shape[1], 1))
        mse = calc_mse( samples, masked_inputs[:,:,0], masked_preds, true_preds, test_inputs, class_idx, 0.005)
        sal[x] = mse.T
        plt.figure()
        
        plt.plot(minmax(sal[x]))
        plt.title('mse')
        plt.show()
        
    np.save("/home/s7thakur/ecresearch-shared/compute1/MIRROR/scores/UCI-" + dataset +"-MIRROR-SaliencyScore-noreconstruction.npy",sal)
    return sal
    # np.save("/rhome/s7thakur/MIRROR/scores/UCI-" + dataset +"-MIRROR-SaliencyScore-noreconstruction.npy",sal)

In [ ]:
sal=explain(x_train, x_test, y_test, y_train,nb_labels, model)

In [ ]:
from scipy.signal import savgol_filter

def eval_insertion_score(test, sal, y_test, dataset, w=11,step=5, save=False,cbar=False):
    X = test
    Y = np.argmax(y_test, axis=1)
#     labels=y_test
    classes = np.unique(Y)
    for c in classes:    
        plt.figure()        
        c_x = X[np.where(Y == int(c))]
        c_sal = sal[np.where(Y==int(c))]
        i=0
        for sa, test_input in zip(c_sal, c_x):
            print(sa[:,int(c)].shape)
            compute_metric_score(test_input, (sa[:,int(c)]), dataset, i, w, step, save,'ins',cbar)
            
            i=i+1
        plt.show()

In [ ]:
from scipy.signal import savgol_filter

def eval_insertion_score_all(test, sal, y_test,  masked_preds_inputs, dataset, w=11,step=5, save=False,cbar=False):
    X = test
#     print(y_test)
#     Y=y_test
    Y = np.argmax(y_test, axis=1)
#     labels=y_test

    classes = np.unique(Y)
#     print(classes)
    for sa, test_input, masked_preds in zip(sal, test, masked_preds_inputs):
        plt.figure()        
        i=0
        # compute ins score for all the classes, bar char of class frequencies, or violin plot
        print('Next test input saliency map')
        for c in classes:    
#             print('class',c)
            s_c = sa[:,int(c)]
#             print(s_c.shape)
            compute_metric_score(test_input,s_c, dataset, i, w, step, save,'ins',cbar, c)
            
        
            i=i+1
        
        plt.show()
        
#         df_preds = pd.DataFrame(np.concatenate([masked_preds_inputs, y_test], axis=0), columns=['Score','Class'])

In [ ]:
ids = np.argmax(masked_preds_inputs,axis=2)
m,n = ids.shape[:2]
I,J = np.ogrid[:m,:n]
pred_labels= (ids).flatten()
pred_scores = (masked_preds_inputs[I,J,ids]).flatten()
inp_ids = [[i]*100 for i in I]
inp_ids = np.array(inp_ids).flatten()
true_labels=[[l]*100 for l in np.argmax(labels, axis=1)]
true_labels = np.array(true_labels).flatten()

df = pd.DataFrame(data=np.column_stack((inp_ids, pred_scores, pred_labels, true_labels)), columns=['Sample','Prediction Score','Prediction Label','True Label'])


In [ ]:
df.head()

In [ ]:
import plotly.express as px

# g = sns.catplot(x="Prediction Label", y="total_bill", kind="violin", inner=None, data=df)
# sns.swarmplot(x="Prediction Label", y="Prediction Score", color="k", size=3, data=df, ax=g.ax)

fig = px.histogram(df, x="Sample", y="Prediction Score", color="Prediction Label",
                   marginal="box")
fig.show()



In [ ]:
# eval_insertion_score_all(test, sal, labels,  masked_preds_inputs, dataset, 5, 5, False,False)
eval_insertion_score(test, sal, labels, dataset, 5, 5, False,False)

In [ ]:
# for different classifiers

# for different datasets

# report average insertion score (AUC score)


In [ ]:
sal[np.where(labels==int(0))[0]].shape

In [ ]:
sal.shape

In [ ]:
Counter(Y_train[1500:1600])

In [ ]:
def get_test_set( x_test, y_test):
    
    test = x_test[1550:1600]
    labels = y_test[1550:1600]
        
    return test,labels
    

In [ ]:
sal=np.load('/home/s7thakur/ecresearch-shared/MIRROR/results/resnet/PhysioNet/mitdb/importance_score.npy', allow_pickle=True)
# sal=sal.item()['importance_scores']

In [ ]:
sal.item()

In [ ]:
# sal = ((sal))
from scipy.signal import savgol_filter

test,labels=get_test_set( X_train,Y_train)
X = test
# Y = np.argmax(labels,axis=1)
Y = labels
classes = np.unique(Y)
print(classes)
print(X.shape, Y.shape, sal.shape)
# mse1 = mse1.reshape(mse1.shape[1], mse1.shape[0])
ind=0
# fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))
for c in classes:
    
#     plt.figure(figsize=(10,5))        
    c_x = X[np.where(Y == c)]
    c_sal = sal[np.where(Y==c)]
    print(c_x.shape, c_sal.shape)
    for sa, test_input in zip(c_sal, c_x):
        sa = savgol_filter(sa[:,c], 5, 1) # window size 51, polynomial order 3
        max_length = 6000
        sa = norm(sa)
        minimum = np.mean(sa)
        
        sa = sa - minimum
        sa = sa / np.max(sa)
        sa= sa * 100
        
        ts = test_input.reshape(1, test.shape[2],1)
        x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
        f = interp1d(range(ts.shape[1]), ts[0, :, 0])
        y = f(x)
        
        f = interp1d(range(ts.shape[1]), sa)
        cas = f(x).astype(int)
#         plt.plot(cas)

        plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
        
    plt.title('class:{}'.format(c), fontsize=20)
        
        
   
    plt.show()

In [ ]:
# sal = ((sal))
from scipy.signal import savgol_filter

# test,labels=get_test_set(X_train,Y_train, X_test,Y_test)
X =X_test
# Y = labels
# classes = np.unique(Y)
Y= Y_test
# print(classes)
# print(X.shape, Y.shape, sal.shape)
# mse1 = mse1.reshape(mse1.shape[1], mse1.shape[0])
ind=0
# fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))
for c in classes:
    
#     plt.figure(figsize=(10,5))        
    c_x = X[np.where(Y == c)]
    c_sal = sal[np.where(Y==c)]
    print(c_x.shape, c_sal.shape)
    for sa, test_input in zip(c_sal, c_x):
        sa = savgol_filter(sa[:,c], 5, 1) # window size 51, polynomial order 3
        max_length = 6000
        sa = norm(sa)
        minimum = np.mean(sa)
        
        sa = sa - minimum
        sa = sa / np.max(sa)
        sa= sa * 100
        
        ts = test_input.reshape(1, test.shape[1],1)
        x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
        f = interp1d(range(ts.shape[1]), ts[0, :, 0])
        y = f(x)
        
        f = interp1d(range(ts.shape[1]), sa)
        cas = f(x).astype(int)
#         plt.plot(cas)

        plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
        
    plt.title('class:{}'.format(c), fontsize=20)
        
        
   
    plt.show()

In [ ]:



# save saliency score visualization 
# sal = np.abs(sal)
from scipy.signal import savgol_filter

X = test
Y = np.argmax(labels,axis=1)
# Y = labels
classes = np.unique(Y)
print(classes)
print(X.shape, Y.shape, sal.shape)
# mse1 = mse1.reshape(mse1.shape[1], mse1.shape[0])
ind=0
fig, axes = plt.subplots( ncols=len(classes), figsize=(20, 4))
# fig.subplots_adjust(hspace=0.5)

for c,ax in zip(classes,axes):
    
#     plt.figure(figsize=(10,5))        
    c_x = X[np.where(Y == c)]
    c_sal = sal[np.where(Y==c)]
    print(c_x.shape, c_sal.shape)
    for sa, test_input in zip(c_sal, c_x):
#         win = len(test_input_)/2
        sa = savgol_filter(sa[:,c], 5, 1) # window size 51, polynomial order 3
        max_length = 6000
#         sa = np.abs(sa)
        minimum = np.mean(sa)
        
        sa = sa - minimum
        sa = sa / np.max(sa)
        sa= sa * 100
        
        ts = test_input.reshape(1, test.shape[1],1)
        x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
        f = interp1d(range(ts.shape[1]), ts[0, :, 0])
        y = f(x)
        
        f = interp1d(range(ts.shape[1]), sa)
        cas = f(x).astype(int)
#         plt.plot(cas)

#         plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
        
#     plt.title('class:{}'.format(c), fontsize=20)
        
        
        
        ax.scatter(x,y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
    ax.set_title('class:{}'.format(c), size=25) 
    ax.set_ylabel('Saliency Score', size=25) #setting the ylabel and font size
    ax.set_xlabel('Index', size=25)
    ax.xaxis.set_tick_params(labelsize=25) #setting the font size of the x axis
    ax.yaxis.set_tick_params(labelsize=25) #setting the font size of the y axis
#         ind = ind+1
    
plt.tight_layout()
# plt.savefig('/rhome/s7thakur/MIRROR/plots/UCI-'+dataset+'-MIRROR-SaliencyScore-run2.eps')
# plt.savefig('/home/s7thakur/ecresearch-shared/compute1/MIRROR/plots/UCI-'+dataset+'-MIRROR-SaliencyScore-no-reconstruction-51.pdf',
#            bbox_inches='tight',rasterized=True)
# plt.savefig('/rhome/s7thakur/MIRROR/plots/UCI-'+dataset+'-MIRROR-SaliencyScore-run2.svg')

plt.show()


In [ ]:
sal.shape

In [ ]:
# saliency for all the combination of classes and inputs

# save saliency score visualization 
sal = (sal)
from scipy.signal import savgol_filter

X = test
# Y = np.argmax(labels,axis=1)
Y = labels
classes = np.unique(Y)
print(classes)
print(X.shape, Y.shape, sal.shape)
# mse1 = mse1.reshape(mse1.shape[1], mse1.shape[0])
ind=0
fig, axes = plt.subplots(nrows=len(classes), ncols=len(classes), figsize=(25, 15))
# fig.subplots_adjust(hspace=0.5)

for c_i in classes:
    
#     plt.figure(figsize=(10,5))   

    c_x = X[np.where(Y == c_i)]
    c_sal = sal[np.where(Y==c_i)]
    print(c_i, c_x.shape, c_sal.shape)
        
    for c_j in classes:
        for sa, test_input in zip(c_sal, c_x):
#             print(sa.shape)
    #         win = len(test_input_)/2
            sa = savgol_filter(sa[:,c_j], 11, 1) # window size 51, polynomial order 3
            max_length = 6000
    #         sa = np.abs(sa)
            minimum = np.mean(sa)
        

            sa = sa - minimum
            sa = sa / np.max(sa)
            sa= sa * 100

            ts = test_input.reshape(1, test.shape[2],1)
            x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
            f = interp1d(range(ts.shape[1]), ts[0, :, 0])
            y = f(x)

            f = interp1d(range(ts.shape[1]), sa)
            cas = f(x).astype(int)
    #         plt.plot(cas)

    #         plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)

    #     plt.title('class:{}'.format(c), fontsize=20)



            axes[c_i,c_j].scatter(x,y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
        axes[c_i,c_j].set_title('True Class:{}, Saliency Map Class:{}'.format(c_i,c_j), size=25) 
        axes[c_i,c_j].set_ylabel('Saliency Score', size=20) #setting the ylabel and font size
        axes[c_i,c_j].set_xlabel('Time (ms)', size=20)
        axes[c_i,c_j].xaxis.set_tick_params(labelsize=15) #setting the font size of the x axis
        axes[c_i,c_j].yaxis.set_tick_params(labelsize=15) #setting the font size of the y axis
    #         ind = ind+1
   
fig.tight_layout()
# plt.savefig('/rhome/s7thakur/MIRROR/plots/UCI-'+dataset+'-MIRROR-SaliencyScore-run2.eps')
# plt.savefig('/home/s7thakur/ecresearch-shared/MIRROR/plots/UCI-PhysioNet-0-1-MIRROR-class-discriminative-SaliencyScore.pdf')
plt.savefig('/home/s7thakur/ecresearch-shared/MIRROR/plots/UCI-PhysioNet-0-1-MIRROR-class-discriminative-SaliencyScore.png',resolution=300)

# plt.savefig('/rhome/s7thakur/MIRROR/plots/UCI-'+dataset+'-MIRROR-SaliencyScore-run2.svg')

# plt.show()

In [ ]:
def auc(arr):
    return (arr.sum() - arr[0]/2 - arr[-1]/2)/(arr.shape[0]-1)

def compute_metric_score(test, sal, dataset, inp_n, w_s=11, step=5,save_to=False,mode='ins',cbar=False, given_clas='None'):
    
    
    save_path = "/home/s7thakur/ecresearch-shared/compute1/MIRROR/plots/"
    W = test.shape[0] # image area
    n_classes = np.unique(labels)
    mode = 'ins'
    inp = test[:,0]
#     sa = np.abs(sal)
    sa = savgol_filter(sal, w_s, 1) # window size 51, polynomial order 3
    minimum = np.mean(sa)

    sa = sa - minimum
    sa = sa / np.max(sa)
    sa= sa * 100

    explanation = sa
#     fig, ax1 = plt.subplots()
#     ax1.plot(inp)
#     ax2 = ax1.twinx()
#     ax2.plot(explanation, color='red')
    
#     plt.show()

    verbose = 1
    
    substrate = np.zeros(inp.shape)
    
    pred = model.predict(inp.reshape(1,W,1))
    n_steps = (W + step - 1) // step
    if mode == 'del':
        title = 'Deletion game'
        ylabel = 'Sample deleted'
        start = inp
        finish = substrate

    elif mode == 'ins':
        title = 'Insertion game'
        ylabel = 'Sample inserted'
        start = substrate
        finish = inp

    scores = np.empty(n_steps + 1)
    clas = np.empty(n_steps + 1)
    
    salient_order = np.flip(np.argsort(explanation))
    cl = np.argmax(pred)
    
    for i in range(n_steps+1):
        
        
        pred = model.predict(start.reshape(1, W, 1))
        if verbose == 2:
            print('{}: {:.3f}'.format(cl, float(pr)))
        scores[i] = pred[0][cl]
        clas[i] = np.argmax(pred[0])
        if verbose == 2 or (verbose == 1 and i == n_steps) or save_to:
            plt.figure(figsize=(14, 4))
            plt.subplot(121)
            plt.title('{} {:.1f}%, Pred Class:{}={:.4f}, Given Class:{}'.format(ylabel, 100 * i / n_steps, cl, scores[i],given_clas))
            plt.axis('off')
#             plt.plot(start)
             
            ################################################################################
            
            max_length = 6000
            ts = test.reshape(1, test.shape[0],1)
            
            x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
            f = interp1d(range(ts.shape[1]), ts[0, :, 0])
            y = f(x)

            f = interp1d(range(ts.shape[1]), sa)
            cas = f(x).astype(int)
            plt.scatter(x,y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)
            if cbar:
                plt.colorbar()
        
            ##################################################################################

            plt.subplot(122)
            plt.plot(np.arange(i+1) / n_steps, scores[:i+1])
            plt.xlim(-0.1, 1.1)
            plt.ylim(0, 1.05)
            plt.fill_between(np.arange(i+1) / n_steps, 0, scores[:i+1], alpha=0.4)
            plt.title('{} AUC Score={:.4f}'.format(title, auc(scores)))
            
            plt.xlabel(ylabel)
            plt.ylabel('Prob Score')

            if save_to:
        
                plt.savefig(save_path + "UCI-"+dataset+'-MIRROR-insertion-input-'+str(inp_n)+'-win-'+str(w_s)+'-class-'+str(cl)+'.pdf')
                plt.close()
            else:
                plt.show()
        if i < n_steps:

            coords = salient_order[step * i:step * (i + 1)]
            start[coords] = finish[coords]
        
    
    return scores
#     print(scores)
#     print('AUC Score=',auc(scores))

In [ ]:
# read datasets from model folders
datasets = pd.read_csv('/rhome/s7thakur/time-series-models/DataSummary.csv')['Name']
inception = glob.glob(os.path.join('/rhome/s7thakur/time-series-models/inception/*'))
resnet = glob.glob(os.path.join('/rhome/s7thakur/time-series-models/resnet/*'))
cnn = glob.glob(os.path.join('/rhome/s7thakur/time-series-models/cnn/*'))
autoencoder = glob.glob(os.path.join('/rhome/s7thakur/time-series-models/autoencoder/*'))
# extrace the dataset names

inception = [f.split('/')[5] for f in inception]
autoencoder = [f.split('/')[5] for f in autoencoder]
resnet = [f.split('/')[5] for f in resnet]
cnn = [f.split('/')[5] for f in cnn]
# create dataframe of the lists
inception = pd.DataFrame(inception)
autoencoder = pd.DataFrame(autoencoder)
resnet = pd.DataFrame(resnet)
cnn = pd.DataFrame(cnn)

In [ ]:
# read datasets from model folders
datasets = pd.read_csv('/rhome/s7thakur/ResNet1D/DataSummary.csv')['Name']
resnet = glob.glob(os.path.join('/rhome/s7thakur/'))
autoencoder = glob.glob(os.path.join('/rhome/s7thakur/ResNet1D/autoencoder/*'))
# extrace the dataset names
autoencoder = [f.split('/')[5] for f in autoencoder]
# resnet = [f.split('/')[5] for f in resnet]
# create dataframe of the lists
autoencoder = pd.DataFrame(autoencoder)
# resnet = pd.DataFrame(resnet)
dir = '/rhome/s7thakur/ResNet1D'

sal_dir = '/rhome/s7thakur/MIRROR/logs/vae/saliency/'

## Saliency

In [ ]:
models = ['resnet', 'cnn', 'inception']
# models = ['resnet']
datasets = ['PhalangesOutlinesCorrect', 'CinCECGTorso','ItalyPowerDemand', 'Trace', 'GunPoint', 'GunPointAgeSpan', 
                            'Strawberry', 'ECGFiveDays', 'TwoLeadECG', 'Chinatown', 'DistalPhalanxOutlineCorrect']

# datasets = ['Trace']
for model in models:
    
    for dataset in datasets:
#         dataset = dataset[0]if model == 'resnet':
        
        x_train, y_train, x_test, y_test = read_dataset(archive_dir, dataset)
        x_train, x_test = reshape(x_train, x_test)
        y_train, y_test = label_encoding(y_train, y_test)

        if x_train.shape[0] < x_test.shape[0]:
            test = x_train[0:50]
            labels = y_train[0:50]

        else:
            test = x_test[0:50]
            labels = y_test[0:50]


        X = test
        Y = np.argmax(labels,axis=1)
        classes = np.unique(Y)

        window_len = x_test.shape[1] - 10
        if window_len%2 == 0:
            window_len = window_len+1
        
        
        sal_path = os.path.join(sal_dir,dataset+'-'+model+'.npy')
        sal = np.load(sal_path)
#         print(sal.shape)
#         sal = np.abs(sal)
        from scipy.signal import savgol_filter

        X = test
        # Y = np.argmax(labels,axis=1)
        classes = np.unique(Y)

        print(X.shape, Y.shape, sal.shape, window_len)

        for c in classes:

            plt.figure(figsize=(10,5))        
            c_x = X[np.where(Y == c)]
            c_sal = sal[np.where(Y==c)]
            print('Dataset:',dataset)
            for sa, test_input in zip(c_sal, c_x):
                plt.plot(sa)
            plt.show()
    
            plt.figure(figsize=(10,5))        
#             c_sal = np.abs(c_sal)
            for sa, test_input in zip(c_sal, c_x):

                
                sa = savgol_filter(sa[:,0], 21, 1) # window size 51, polynomial order 3
                
                max_length = 4000
                minimum = np.mean(sa)

                sa = sa - minimum
                sa = sa / np.max(sa)
                sa= sa * 100

                ts = test_input.reshape(1, test.shape[1],1)
                x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
                f = interp1d(range(ts.shape[1]), ts[0, :, 0])
                y = f(x)

                f = interp1d(range(ts.shape[1]), sa)
                cas = f(x).astype(int)
        #         plt.plot(cas)
                plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=10, vmin=0, vmax=100, linewidths=0.0)

                plt.title('Model:{}, Dataset:{}, Class:{}'.format(model, dataset, c), fontsize=20)
            plt.show()

In [ ]:
# models = ['resnet', 'cnn', 'inception']
models = ['resnet']
datasets = ['PhalangesOutlinesCorrect', 'CinCECGTorso','ItalyPowerDemand', 'Trace', 'GunPoint', 'GunPointAgeSpan', 
                            'Strawberry', 'ECGFiveDays', 'TwoLeadECG', 'Chinatown', 'DistalPhalanxOutlineCorrect']


for model in models:

    for dataset in datasets:
#         dataset = dataset[0]if model == 'resnet':
         
        x_train, y_train, x_test, y_test = read_dataset(archive_dir, dataset)
        x_train, x_test = reshape(x_train, x_test)
        
        if(x_train.shape[0] == 0): continue
        if(x_test.shape[0] == 0): continue
        
#         y_train, y_test = label_encoding(y_train, y_test)

        # transform to binary labels
        enc = sklearn.preprocessing.OneHotEncoder()
        enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
        y_train_binary = enc.transform(y_train.reshape(-1, 1)).toarray()

        if x_train.shape[0] < x_test.shape[0]:
            
            if x_train.shape[0] > 50:
                test = x_train[0:50]
                labels = y_train[0:50]
            elif x_train.shape[0] >0:
                test = x_train
                labels = y_train
            else: 
                print('Length zero')
                continue
        else:
            if x_test.shape[0]>50:
                test = x_test[0:50]
                labels = y_test[0:50]
            elif x_test.shape[0]>0:
                test = x_test
                labels = y_test
            else:
                print('Length zero')
                continue
            
#         X = test.reshape(test.shape[0], test.shape[1], 1)
        
#         Y = np.argmax(labels,axis=1)
        classes = np.unique(np.concatenate((y_train, y_test), axis=0))
        print(classes)
        model = keras.models.load_model(
            '/rhome/s7thakur/ResNet1D/resnet' + '/' + dataset + '/best_model.hdf5')

        print(labels.shape, test.shape)
        # filters
        w_k_c = model.layers[-1].get_weights()[0]  # weights for each filter k for each class c

        # the same input
        new_input_layer = model.inputs
        # output is both the original as well as the before last layer
        new_output_layer = [model.layers[-3].output, model.layers[-1].output]

        new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)


        for c in classes:
            plt.figure(figsize=(10,5))
            count = 0
            c_x_train = test[np.where(labels == c)]
            if c_x_train.shape[0] == 0: continue
#             print(c_x_train.shape)
            for ts in c_x_train:
                ts = ts.reshape(1, -1, 1)
                [conv_out, predicted] = new_feed_forward([ts])
        #         print(conv_out.shape)
                pred_label = np.argmax(predicted)
                orig_label = np.argmax(enc.transform([[c]]))
                if pred_label == orig_label:
                    cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
                    for k, w in enumerate(w_k_c[:, orig_label]):
                        cas += w * conv_out[0, :, k]
        #                 print(conv_out[0, :, k].shape, w)
                    minimum = np.min(cas)

                    cas = cas - minimum

                    cas = cas / max(cas)
                    cas = cas * 100
                    
                    x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
                    # linear interpolation to smooth
                    f = interp1d(range(ts.shape[1]), ts[0, :, 0])
                    y = f(x)
                    # if (y < -2.2).any():
                    #     continue
                    f = interp1d(range(ts.shape[1]), cas)
                    cas = f(x).astype(int)
                    plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
                    if dataset_name == 'Gun_Point':
                        if c == 1:
                            plt.yticks([-1.0, 0.0, 1.0, 2.0])
                        else:
                            plt.yticks([-2, -1.0, 0.0, 1.0, 2.0])
                    count += 1
                
            cbar = plt.colorbar()
            plt.show()

### CAM generated saliency

In [ ]:
model.

In [ ]:
model.

In [ ]:
# root_dir = '/rhome/s7thakur/'
# root_dir = '/home/s7thakur/ecresearch-shared/compute1'

# classifier = 'ResNet1D/resnet'
# archive_name = 'UCRArchive/UCRArchive_2018'
# dataset_name = 'TwoLeadECG'


# max_length = 2000
# x_train, y_train, x_test, y_test = read_dataset(os.path.join(root_dir, archive_name), dataset_name)

# plt.plot()
# for x in x_train[0:10]:
#     plt.plot(x)
# plt.show()
# # transform to binary labels
# enc = sklearn.preprocessing.OneHotEncoder()
# enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
# y_train_binary = enc.transform(y_train.reshape(-1, 1)).toarray()

# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# model = keras.models.load_model(
#     root_dir + '/' + classifier + '/' + dataset_name + '/best_model.hdf5')

# filters
w_k_c = model.layers[-1].get_weights()[0]  # weights for each filter k for each class c
print(y_train.shape, x_train.shape)
# the same input
new_input_layer = model.inputs
# output is both the original as well as the before last layer
new_output_layer = [model.layers[-3].output, model.layers[-1].output]

new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

classes = np.unique(y_train)

for c in classes:
    plt.figure(figsize=(10,5))
    count = 0
    c_x_train = x_train[np.where(y_train == c)]
    for ts in c_x_train:
        ts = ts.reshape(1, -1, 1)
        [conv_out, predicted] = new_feed_forward([ts])
#         print(conv_out.shape)
        pred_label = np.argmax(predicted)
        orig_label = np.argmax(enc.transform([[c]]))
        if pred_label == orig_label:
            cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
            for k, w in enumerate(w_k_c[:, orig_label]):
                cas += w * conv_out[0, :, k]
#                 print(conv_out[0, :, k].shape, w)

            minimum = np.min(cas)

            cas = cas - minimum

            cas = cas / max(cas)
            cas = cas * 100
#             print(ts[0,:,0].shape, cas.shape)
            
#             compute_metric_score(ts[0,:,:], np.abs(cas),"Trace",0,5,5)
            x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
            # linear interpolation to smooth
            f = interp1d(range(ts.shape[1]), ts[0, :, 0])
            y = f(x)
            # if (y < -2.2).any():
            #     continue
            f = interp1d(range(ts.shape[1]), cas)
            cas = f(x).astype(int)
            plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
            if dataset_name == 'Gun_Point':
                if c == 1:
                    plt.yticks([-1.0, 0.0, 1.0, 2.0])
                else:
                    plt.yticks([-2, -1.0, 0.0, 1.0, 2.0])
            count += 1

    cbar = plt.colorbar()
    plt.show()

In [ ]:
root_dir = '/rhome/s7thakur/'
root_dir = '/home/s7thakur/ecresearch-shared/compute1'

classifier = 'ResNet1D/resnet'
archive_name = 'UCRArchive/UCRArchive_2018'
dataset_name = 'TwoLeadECG'


# max_length = 2000
# x_train, y_train, x_test, y_test = read_dataset(os.path.join(root_dir, archive_name), dataset_name)

# plt.plot()
# for x in x_train[0:10]:
#     plt.plot(x)
# plt.show()
# # transform to binary labels
# enc = sklearn.preprocessing.OneHotEncoder()
# enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
# y_train_binary = enc.transform(y_train.reshape(-1, 1)).toarray()

# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

models = keras.models.load_model(
    root_dir + '/' + classifier + '/' + dataset_name + '/best_model.hdf5')

# filters
print(models.layers[-1])
print(models.layers[-3])
# w_k_c = model.layers[-1].get_weights()[0]  # weights for each filter k for each class c
# print(y_train.shape, x_train.shape)
# # the same input
# new_input_layer = model.inputs
# # output is both the original as well as the before last layer
# new_output_layer = [model.layers[-3].output, model.layers[-1].output]

# new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

# classes = np.unique(y_train)

# for c in classes:
#     plt.figure(figsize=(10,5))
#     count = 0
#     c_x_train = x_train[np.where(y_train == c)]
#     for ts in c_x_train:
#         ts = ts.reshape(1, -1, 1)
#         [conv_out, predicted] = new_feed_forward([ts])
# #         print(conv_out.shape)
#         pred_label = np.argmax(predicted)
#         orig_label = np.argmax(enc.transform([[c]]))
#         if pred_label == orig_label:
#             cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
#             for k, w in enumerate(w_k_c[:, orig_label]):
#                 cas += w * conv_out[0, :, k]
# #                 print(conv_out[0, :, k].shape, w)

#             minimum = np.min(cas)

#             cas = cas - minimum

#             cas = cas / max(cas)
#             cas = cas * 100
# #             print(ts[0,:,0].shape, cas.shape)
            
# #             compute_metric_score(ts[0,:,:], np.abs(cas),"Trace",0,5,5)
#             x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
#             # linear interpolation to smooth
#             f = interp1d(range(ts.shape[1]), ts[0, :, 0])
#             y = f(x)
#             # if (y < -2.2).any():
#             #     continue
#             f = interp1d(range(ts.shape[1]), cas)
#             cas = f(x).astype(int)
#             plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
#             if dataset_name == 'Gun_Point':
#                 if c == 1:
#                     plt.yticks([-1.0, 0.0, 1.0, 2.0])
#                 else:
#                     plt.yticks([-2, -1.0, 0.0, 1.0, 2.0])
#             count += 1

#     cbar = plt.colorbar()
#     plt.show()

In [ ]:
models = ['resnet', 'cnn', 'inception']

datasets = dataset_names = ['PhalangesOutlinesCorrect', 'CinCECGTorso','ItalyPowerDemand', 'Trace', 'GunPoint', 'GunPointAgeSpan', 
                            'Strawberry', 'ECGFiveDays', 'TwoLeadECG', 'Chinatown', 'DistalPhalanxOutlineCorrect']


for model in models:

    for dataset in autoencoder.values:
#         dataset = dataset[0]if model == 'resnet':
        if dataset not in resnet.values:
            continue
        if model == 'cnn':
            if dataset not in cnn.values:
                continue
        if model == 'inception':
            if dataset not in inception.values:
                continue
        print(dataset)
        
        x_train, y_train, x_test, y_test = read_dataset(archive_dir, dataset)
        x_train, x_test = reshape(x_train, x_test)
        y_train, y_test = label_encoding(y_train, y_test)

        if x_train.shape[0] < x_test.shape[0]:
            test = x_train[0:50]
            labels = y_train[0:50]

        else:
            test = x_test[0:50]
            labels = y_test[0:50]


        X = test
        Y = np.argmax(labels,axis=1)
        classes = np.unique(Y)


        window_len = x_test.shape[1] - 10
        if window_len%2 == 0:
            window_len = window_len+1
        
        
        sal_path = os.path.join(dir, dataset+'-'+model+'.npy')
        sal = np.load(sal_path)
#         print(sal.shape)
#         sal = np.abs(sal)
        from scipy.signal import savgol_filter

        X = test
        # Y = np.argmax(labels,axis=1)
        classes = np.unique(Y)

        print(X.shape, Y.shape, sal.shape, window_len)

        for c in classes:

            plt.figure(figsize=(10,5))        
            c_x = X[np.where(Y == c)]
            c_sal = sal[np.where(Y==c)]
            print('Dataset:',dataset)
            for sa, test_input in zip(c_sal, c_x):
                plt.plot(sa)
            plt.show()
    
            plt.figure(figsize=(10,5))        
            c_sal = np.abs(c_sal)
            for sa, test_input in zip(c_sal, c_x):

                sa = savgol_filter(sa[:,0], 71, 1) # window size 51, polynomial order 3
                max_length = 4000
                minimum = np.mean(sa)

                sa = sa - minimum
                sa = sa / np.max(sa)
                sa= sa * 100

                ts = test_input.reshape(1, test.shape[1],1)
                x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
                f = interp1d(range(ts.shape[1]), ts[0, :, 0])
                y = f(x)

                f = interp1d(range(ts.shape[1]), sa)
                cas = f(x).astype(int)
        #         plt.plot(cas)
                plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=20, vmin=0, vmax=100, linewidths=0.0)

                plt.title('class:{}'.format(c), fontsize=20)
            plt.show()

In [ ]:
##### root_dir = '/rhome/s7thakur'
classifier = 'ResNet1D'
archive_name = 'UCRArchive/UCRArchive_2018'
dataset_name = 'Trace'


max_length = 2000
x_train, y_train, x_test, y_test = read_dataset(os.path.join(root_dir, archive_name), dataset_name)

plt.plot()
for x in x_train[0:10]:
    plt.plot(x)
plt.show()
# transform to binary labels
enc = sklearn.preprocessing.OneHotEncoder()
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train_binary = enc.transform(y_train.reshape(-1, 1)).toarray()

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

model = keras.models.load_model(
    root_dir + '/' + classifier + '/' + dataset_name + '/best_model.hdf5')

# filters
w_k_c = model.layers[-1].get_weights()[0]  # weights for each filter k for each class c

# the same input
new_input_layer = model.inputs
# output is both the original as well as the before last layer
new_output_layer = [model.layers[-3].output, model.layers[-1].output]

new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

classes = np.unique(y_train)

for c in classes:
    plt.figure(figsize=(10,5))
    count = 0
    c_x_train = x_train[np.where(y_train == c)]
    for ts in c_x_train:
        ts = ts.reshape(1, -1, 1)
        [conv_out, predicted] = new_feed_forward([ts])
#         print(conv_out.shape)
        pred_label = np.argmax(predicted)
        orig_label = np.argmax(enc.transform([[c]]))
        if pred_label == orig_label:
            cas = np.zeros(dtype=np.float, shape=(conv_out.shape[1]))
            for k, w in enumerate(w_k_c[:, orig_label]):
                cas += w * conv_out[0, :, k]
#                 print(conv_out[0, :, k].shape, w)

            minimum = np.min(cas)

            cas = cas - minimum

            cas = cas / max(cas)
            cas = cas * 100

            x = np.linspace(0, ts.shape[1] - 1, max_length, endpoint=True)
            # linear interpolation to smooth
            f = interp1d(range(ts.shape[1]), ts[0, :, 0])
            y = f(x)
            # if (y < -2.2).any():
            #     continue
            f = interp1d(range(ts.shape[1]), cas)
            cas = f(x).astype(int)
            plt.scatter(x=x, y=y, c=cas, cmap='jet', marker='.', s=2, vmin=0, vmax=100, linewidths=0.0)
            if dataset_name == 'Gun_Point':
                if c == 1:
                    plt.yticks([-1.0, 0.0, 1.0, 2.0])
                else:
                    plt.yticks([-2, -1.0, 0.0, 1.0, 2.0])
            count += 1

    cbar = plt.colorbar()
    plt.show()

In [ ]:
models.layers

In [ ]:
models.summary()

In [ ]:
x_train.shape

In [ ]:
inception = [f.split('/')[5] for f in inception]
autoencoder = [f.split('/')[5] for f in autoencoder]
resnet = [f.split('/')[5] for f in resnet]
cnn = [f.split('/')[5] for f in cnn]

In [ ]:
inception = pd.DataFrame(inception)
autoencoder = pd.DataFrame(autoencoder)
resnet = pd.DataFrame(resnet)
cnn = pd.DataFrame(cnn)

In [ ]:

import re
my_str = "hey123: th~!ere {} for !! example 20"
keywords = re.sub('[^a-zA-Z0-9 \n\.]', '', my_str)
keywords = re.sub('\s+',' ',keywords)

In [ ]:
print(my_new_string)

In [ ]:
keywords = list(keywords.split(' '))
keywords = '-'.join(keywords)
print(keywords)